# BAI-Assistent

Grundidee: Hilft bei der Erstellung von Zusammenfassungen, welche auf Basis unseren Zusammenfassungen und Vorlesungsfolien die Antworten generiert

#### Unsere Problemstellung
Während der Prüfungsvorbereitung sind vor allem Erstsemester Studenten überfordert, wie man beim Lernen vorgehen kann. Daher haben wir es als Lücke vor allem im BAI-Studiengang erkannt.

#### Use Cases: 
Ich möchte, dass mir der Lernassistent mir Fachbegriffe in ML und Einführung KI erklärt
Ich möchte gut auf die Prüfungen durch den Lernassistenten vorbereitet werden
-	Ich möchte Prüfungsfragen erhalten
-	Ich möchte, dass es Merksätze gibt
-	Ich möchte, dass die Erklärungen einfach sind
-	Ich möchte Hilfe/Beratung erhalten, wie ich mein Cheat Sheet gemäss Stoffabgrenzung aufstellen kann
Ich möchte schnelle und unlimitierte Antworten
(Ich möchte Prüfungsfragen vom Chatbot erhalten, damit ich mich gut auf die Prüfung vorbereiten kann)

##### Zielgruppe: 
BAI-Studenten im ersten Studienjahr, die Maschinelles Lernen und Einführung in die Künstliche Intelligenz belegen

##### KPIs: 
•	Antwortzeit < 5 Sekunden
•	Prüfungsnutzen > 70 % finden Quiz hilfreich
•	Fachliche Korrektheit >85%

Unsere Erwartungen: Fachbegriffe fragen, Unterschied zwischen Supervised und Unsupervised Learning, Was ist One Hot Encoding
Inhalte für den KI-Assistenten: Folien Unterricht, Zusammenfassungen, Stoffabgrenzung


#### Zusammenführung LLM und API

In [1]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

# --- Stelle sicher, dass dieser Key existiert ---
assert "GROQ_API_KEY_BAI" in os.environ, "GROQ_API_KEY fehlt in den Env Vars!"

# --- Initialisiere LLM explizit für OpenRouter ---
llm = ChatOpenAI(
    model="openai/gpt-oss-120b",   
    api_key=os.environ["GROQ_API_KEY_BAI"],
    base_url="https://api.groq.com/openai/v1",
    temperature=0.3,
)

print("Sende Test-Ping...")
try:
    msg = llm.invoke("Sag exakt: pong")
    print("Antworttyp:", type(msg))
    # msg ist i.d.R. ein AIMessage – gib Inhalt sicher aus:
    print("Inhalt:", getattr(msg, "content", msg))
except Exception as e:
    print("FEHLER beim LLM-Aufruf:", repr(e))


Sende Test-Ping...
Antworttyp: <class 'langchain_core.messages.ai.AIMessage'>
Inhalt: pong


In [2]:
import os
import langchain
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI


#LLM_MODEL = "openai/gpt-oss-20b:free"
LLM_MODEL = "openai/gpt-oss-120b"
LLM_TEMPERATURE = 0.4
BASE_URL = "https://api.groq.com/openai/v1"
OPENROUTER_API_KEY = os.getenv("GROQ_API_KEY_BAI")
USER_PROMPT="Ich verstehe GenAI nicht, kannst du das mir einfach erklären?"

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


llm = ChatOpenAI(
    model=LLM_MODEL,
    temperature=LLM_TEMPERATURE,
    base_url=BASE_URL,
    api_key=OPENROUTER_API_KEY,
)

print(type(llm))


<class 'langchain_openai.chat_models.base.ChatOpenAI'>


#### Kurz sicherstellen, ob API Key funktioniert

In [4]:
try:
    print(llm.invoke("Sag nur: pong").content)
except Exception as e:
    print(repr(e))

# Test 2: Env-Variablen sichtbar?
import os
print("OPENAI_API_KEY" in os.environ, os.environ.get("OPENAI_BASE_URL"))
print("GROQ_API_KEY_BAI" in os.environ)
print("OPENROUTER_API_KEY" in os.environ)

pong
False None
True
False


#### ChatPrompt Template

In [5]:
from langchain_core.prompts import ChatPromptTemplate

FWO_PROMPT = ChatPromptTemplate.from_messages([
    ("system",
     "Du bist der FWO-Assistent (Fachschaft Wirtschaft Olten, FHNW). "
     "Bevor du schreibst, beziehst du dich IMMER auf: "
     "1) Kampagnenkonzept (Ziel, Kernbotschaft, Mehrwert) "
     "2) Redaktionsplan (Phase, wiederkehrende Formate, Timing). "
     "Vermeide Genderung und gebrauche bspw. Lehrperson anstatt Lehrer*innen "
     "Generiere jeweils immer zwei Vorschläge zu jeder Plattform (Instagram, WhatsApp, LinkedIn)"
     "Vermeide Floskeln, schreibe aktiv und konkret."
     "If you are unsure or the answer isn't in the context, say that you don't know.\n\n"
     "CONTEXT: \n {context}"
    ),
    ("human","{question}"),
])


#### PDF Loader

In [6]:
from langchain_community.document_loaders import PyPDFLoader

# List of PDF file paths

pdf_path = "Order pdf/SoMe Konzept.pdf"

loader = PyPDFLoader(str(pdf_path))
pages = loader.load()


# Load all documents
# all_pages_pdf = []
# for pdf in pdf_files:
#     loader = PyPDFLoader(pdf)
#     pages = loader.load()
#     all_pages_pdf.extend(pages)

#print(f"Loaded {len(all_pages_pdf)} pages from {len(pdf_files)} pdf documents.")


ValueError: File path Order pdf/SoMe Konzept.pdf is not a valid file or url

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# get both websites and pdfs together
all_docs = pages

# define the splitter and strategy
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
splits = splitter.split_documents(all_docs)

In [ ]:
import numpy as np

lengths = [len(s.page_content) for s in splits]
print(f"Initial documents: {len(all_docs)}")
print(f"Total chunks: {len(splits)}")
print(f"Avg length: {np.mean(lengths):.1f}")
print(f"Min: {np.min(lengths)}, Max: {np.max(lengths)}")

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [ ]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
    normalize_L2=True
)

vector_store.add_documents(documents=splits)

In [ ]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [ ]:
docs = retriever.get_relevant_documents("Hashtags für Instagram")
for i, d in enumerate(docs, 1):
    print(f"--- Retrieved doc {i} ---")
    print(d.page_content[:400], "...\n")

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

# chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | FWO_PROMPT
#     | llm
#     | output_parser
# )

chain = (
{
    "context": retriever,
    "question": RunnablePassthrough(),
}
    | FWO_PROMPT
    | llm
    | StrOutputParser()
)


In [ ]:
# from langchain_core.output_parsers import StrOutputParser

# USER_PROMPT="Generiere mir einen Post für die PubTour am 16. Oktober"

# result = chain.invoke("Generiere mir einen Post für die PubTour am 16. Oktober")
# print(result)

result = chain.invoke("Generiere mir einen Post für die PubTour am 16. Oktober")
print(result)

#result = chain.invoke(user_prompt)
#print(result)

## UI

In [ ]:
import gradio as gr

def answer(question: str) -> str:
    # Kein LangSmith, einfach direkt die Chain ausführen
    try:
        response = chain.invoke(question)
        return response
    except Exception as e:
        return f"⚠️ Fehler bei der Verarbeitung: {e}"

# --- Gradio UI ---
demo = gr.Interface(
    fn=answer,
    inputs=gr.Textbox(label="Question", placeholder="Type your question here..."),
    outputs=gr.Textbox(label="Answer", lines=10),
    title="FWO Chatbot",
    description="Write a social media post based on the context provided.",
)

# if __name__ == "__main__":
demo.launch(share=True)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=56a0a349-7f2e-43e5-8d52-5ded467f6e9c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>